In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

def convert_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(date_str, format='%m/%d/%y %H:%M')
# Leer el archivo CSV y crear un DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/jdanieln/UNIR_Investigacion_IA_Backend/main/data/data.csv')
print("DF ORIGINAL")
df.info()
selected_columns = ["Product", "Quantity Ordered", "Order Date"]
new_df = df[selected_columns]
# Usando dropna sin especificar subset para eliminar filas con valores nulos en todas las columnas
new_df = new_df.dropna(how='all')


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("DF NUEVO")
new_df.info()
new_df['Quantity Ordered'] = pd.to_numeric(new_df['Quantity Ordered'], errors='coerce')
new_df = new_df.dropna(how='all')
new_df.info()
#print(new_df.head(10))
num_rows_before = new_df.shape[0]
# Eliminar filas con datos faltantes
new_df.dropna(inplace=True)
# Obtener el número de filas después de la eliminación
print(new_df.head(10))
num_rows_after = new_df.shape[0]

# Calcular la cantidad de filas eliminadas
num_rows_deleted = num_rows_before - num_rows_after

print("Número de filas eliminadas:", num_rows_deleted)


# Convertir la columna 'Order Date' a formato de fecha

new_df['Order Date'] = new_df['Order Date'].apply(convert_date)



In [ ]:
new_df['Year'] = new_df['Order Date'].dt.year

# Crear una columna para el mes
new_df['Month'] = new_df['Order Date'].dt.month

# Crear una columna para el día
new_df['Day'] = new_df['Order Date'].dt.day

# Crear una columna para la hora
new_df['Hour'] = new_df['Order Date'].dt.hour

# Crear una columna para el minuto
new_df['Minute'] = new_df['Order Date'].dt.minute

print("ULTIMO")
new_df.info()

In [ ]:
print(new_df.describe())
print(new_df.describe(include=['O']))

In [ ]:
unique_products = new_df['Product'].unique()
print(unique_products)

In [ ]:
# Supongamos que seleccionamos el año y mes deseados
year_selected = 2019
month_selected = 1
N = 5  # Puedes cambiar este valor a la cantidad de productos que desees obtener

# Filtrar los datos para el año y mes seleccionados
filtered_data = new_df[(new_df['Year'] == year_selected) & (new_df['Month'] == month_selected)]

# Agrupar los datos filtrados por producto y sumar las cantidades vendidas
product_sales_selected = filtered_data.groupby('Product')['Quantity Ordered'].sum().reset_index()

# Ordenar los productos por la cantidad total vendida (de mayor a menor) para el año y mes seleccionados
most_sold_products_selected = product_sales_selected.sort_values(by='Quantity Ordered', ascending=False).head(N)

print(f"Para el año {year_selected} y el mes {month_selected}:")
print(f"Los {N} productos más vendidos son:")
for index, row in most_sold_products_selected.iterrows():
    print("Producto:", row['Product'])
    print("Cantidad vendida:", row['Quantity Ordered'])
    print("---")

In [ ]:
N = 5  # Puedes cambiar este valor a la cantidad de productos que desees obtener

# Agrupar los datos de todo el DataFrame por producto y sumar las cantidades vendidas
product_sales_total = new_df.groupby('Product')['Quantity Ordered'].sum().reset_index()

# Ordenar los productos por la cantidad total vendida (de mayor a menor) en todo el DataFrame
most_sold_products_total = product_sales_total.sort_values(by='Quantity Ordered', ascending=False).head(N)

print("\nEn todo el DataFrame:")
print(f"Los {N} productos más vendidos son:")
for index, row in most_sold_products_total.iterrows():
    print("Producto:", row['Product'])
    print("Cantidad vendida:", row['Quantity Ordered'])
    print("---")

In [ ]:
N = 5  # Puedes cambiar este valor a la cantidad de productos que desees obtener

# Supongamos que deseamos encontrar el producto más vendido dentro de un rango de fechas específico
start_date = '2019-01-01'
end_date = '2019-02-25'

# Convierte las fechas a objetos datetime
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filtra los datos dentro del rango de fechas
filtered_data = new_df[(new_df['Order Date'] >= start_date) & (new_df['Order Date'] <= end_date)]

# Agrupa los datos filtrados por producto y sumar las cantidades vendidas
product_sales_range = filtered_data.groupby('Product')['Quantity Ordered'].sum().reset_index()

# Ordena los productos por la cantidad total vendida (de mayor a menor) dentro del rango de fechas
most_sold_products_range = product_sales_range.sort_values(by='Quantity Ordered', ascending=False).head(N)

print(f"Para el rango de fechas desde {start_date} hasta {end_date}:")
print(f"Los {N} productos más vendidos son:")
for index, row in most_sold_products_range.iterrows():
    print("Producto:", row['Product'])
    print("Cantidad vendida:", row['Quantity Ordered'])
    print("---")

In [ ]:
# SalesHeatmap: Muestra el mapa de calor de ventas por mes y día de la semana.

# Extraer el mes y el día de la semana de la columna 'Order Date'
new_df['Month'] = new_df['Order Date'].dt.month
new_df['Day of Week'] = new_df['Order Date'].dt.dayofweek  # 0: Lunes, 1: Martes, ..., 6: Domingo

# Crear una tabla pivote para la cantidad vendida por mes y día de la semana
heatmap_data = new_df.pivot_table(index='Month', columns='Day of Week', values='Quantity Ordered', aggfunc='sum')

# Crear un mapa de calor
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', linewidths=.5)
plt.title('Mapa de Calor de Ventas por Mes y Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Mes')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'])
plt.yticks(rotation=0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# SalesTrends : Muestra las tendencias de ventas a lo largo del tiempo en un gráfico de líneas o serie temporal.
# Supongamos que tienes un DataFrame llamado 'new_df' con las columnas 'Order Date' y 'Quantity Ordered'

# Preparación de los datos
new_df['Order Date'] = pd.to_datetime(new_df['Order Date'])  # Asegúrate de que la columna 'Order Date' sea de tipo datetime

# Gráfico de línea para mostrar las tendencias de ventas a lo largo del tiempo
plt.figure(figsize=(12, 6))
plt.plot(new_df['Order Date'], new_df['Quantity Ordered'], color='blue', label='Cantidad Vendida')
plt.xlabel('Fecha')
plt.ylabel('Cantidad Vendida')
plt.title('Tendencias de Ventas a lo Largo del Tiempo')
plt.legend()

# Mostrar el gráfico
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Ejemplo de histograma para la cantidad de productos vendidos
plt.hist(new_df['Quantity Ordered'], bins=20, edgecolor='k')
plt.xlabel('Cantidad Vendida')
plt.ylabel('Frecuencia')
plt.title('Distribución de Cantidad Vendida')
plt.show()

In [ ]:

#ProductFrequency: Muestra el análisis de frecuencia de productos en forma de un gráfico de barras.

# Calcular la frecuencia de productos
product_frequency = new_df['Product'].value_counts()

# Crear un gráfico de barras para mostrar la frecuencia de productos
plt.figure(figsize=(12, 6))
product_frequency.plot(kind='bar', color='blue')
plt.xlabel('Producto')
plt.ylabel('Frecuencia')
plt.title('Análisis de Frecuencia de Productos')
plt.xticks(rotation=90)  # Rotar las etiquetas del eje X para mayor legibilidad

# Mostrar el gráfico de barras
plt.grid(axis='y')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Ejemplo de diagrama de caja para la cantidad vendida
sns.boxplot(x='Product', y='Quantity Ordered', data=new_df)
plt.xticks(rotation=90)
plt.xlabel('Producto')
plt.ylabel('Cantidad Vendida')
plt.title('Diagrama de Caja de Cantidad Vendida por Producto')
plt.show()

In [ ]:
# Ejemplo de gráfico de barras para mostrar los productos más vendidos
top_products = new_df['Product'].value_counts().head(10)
sns.barplot(x=top_products.index, y=top_products.values)
plt.xticks(rotation=90)
plt.xlabel('Producto')
plt.ylabel('Cantidad Vendida')
plt.title('Productos Más Vendidos')
plt.show()

In [ ]:
# Supongamos que tienes un DataFrame llamado 'data' con las columnas 'Order Date' y 'Quantity Ordered'

# Extraer la hora del día y el día de la semana de la columna 'Order Date'
new_df['Hour'] = new_df['Order Date'].dt.hour
new_df['Day of Week'] = new_df['Order Date'].dt.dayofweek  # 0: Lunes, 1: Martes, ..., 6: Domingo

# Crear una tabla pivote para la cantidad vendida por hora y día de la semana
heatmap_data =new_df.pivot_table(index='Hour', columns='Day of Week', values='Quantity Ordered', aggfunc='sum')

# Crear un mapa de calor
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', linewidths=.5)
plt.title('Mapa de Calor de Cantidad Vendida por Hora del Día y Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Hora del Día')
plt.show()

In [6]:

plt.figure(figsize=(12, 6))

# Gráfico de línea para la cantidad vendida a lo largo del tiempo
plt.plot(new_df['Order Date'], new_df['Quantity Ordered'], color='blue', label='Cantidad Vendida')
plt.xlabel('Fecha')
plt.ylabel('Cantidad Vendida')
plt.title('Tendencia de la Cantidad Vendida a lo largo del Tiempo')
plt.legend()

# Mostrar la figura
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Supongamos que tienes un DataFrame llamado 'new_df' con las columnas 'Product' y 'Quantity Ordered'

# Desglose de la demanda por producto
plt.figure(figsize=(12, 6))

# Utiliza seaborn para crear un gráfico de barras
sns.barplot(x='Product', y='Quantity Ordered', data=new_df, ci=None, palette='viridis')
plt.xticks(rotation=90)
plt.xlabel('Producto')
plt.ylabel('Cantidad Vendida')
plt.title('Desglose de la Demanda por Producto')

# Mostrar el gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Supongamos que tienes un DataFrame llamado 'new_df' con las columnas 'Product', 'Order Date' y 'Quantity Ordered'

# Selecciona un producto específico para el historial de ventas
producto_seleccionado = 'iPhone'  # Reemplaza con el nombre del producto que deseas analizar

# Filtra los datos para el producto seleccionado
historial_ventas_producto = new_df[new_df['Product'] == producto_seleccionado]

# Preparación de los datos
historial_ventas_producto['Order Date'] = pd.to_datetime(historial_ventas_producto['Order Date'])
historial_ventas_producto = historial_ventas_producto.set_index('Order Date')

# Gráfico de línea para mostrar el historial de ventas
plt.figure(figsize=(12, 6))
sns.lineplot(x=historial_ventas_producto.index, y=historial_ventas_producto['Quantity Ordered'], label='Cantidad Vendida', color='blue')
plt.xlabel('Fecha')
plt.ylabel('Cantidad Vendida')
plt.title(f'Historial de Ventas de "{producto_seleccionado}"')
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.show()

In [ ]:

# Agrupar los datos por producto y día para obtener la cantidad total vendida por día
daily_sales = new_df.groupby(['Product', pd.Grouper(key='Order Date', freq='D')])['Quantity Ordered'].sum().reset_index()

# Crea características adicionales a partir de la fecha
daily_sales['DayOfWeek'] = daily_sales['Order Date'].dt.dayofweek
daily_sales['Month'] = daily_sales['Order Date'].dt.month


# Preparación de secuencias
sequence_length = 7  # Longitud de la secuencia de tiempo
num_features = len(daily_sales['Product'].unique())

# Transforma los datos en secuencias de tiempo
X = []
y = []
for product in daily_sales['Product'].unique():
    product_data = daily_sales[daily_sales['Product'] == product]['Quantity Ordered'].values
    for i in range(len(product_data) - sequence_length):
        X.append(product_data[i:i+sequence_length])
        y.append(product_data[i+sequence_length])

X = np.array(X)
y = np.array(y)

# División de datos en conjuntos de entrenamiento y prueba
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Reshape de los datos para que sean compatibles con la entrada de la red LSTM
X_train = X_train.reshape(-1, sequence_length, 1)
X_test = X_test.reshape(-1, sequence_length, 1)

# Construcción del modelo
model = keras.Sequential([
    layers.LSTM(64, input_shape=(sequence_length, 1)),
    layers.Dense(1)
])

# Compilación y entrenamiento del modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predicciones
predictions = model.predict(X_test)

# Evaluación del modelo
mse = np.mean((predictions - y_test)**2)
print("Error cuadrático medio:", mse)

In [ ]:
new_df = new_df.drop('Order Date', axis=1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.regularizers import l2
from keras.metrics import MeanSquaredError
from keras.layers import SimpleRNN


# Codificar la variable categórica 'Product'
label_encoder = LabelEncoder()
new_df['Product'] = label_encoder.fit_transform(new_df['Product'])

# Dividir los datos en características (X) y la variable objetivo (y)
X = new_df.drop('Quantity Ordered', axis=1)
y = new_df['Quantity Ordered']

# Escalar las características para normalizarlas
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reformatear los datos para una secuencia temporal (asumiendo que cada fila representa una secuencia temporal)
X = X.reshape(X.shape[0], 1, X.shape[1])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Construir un modelo más simple con una capa SimpleRNN
model = Sequential()
model.add(SimpleRNN(32, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dense(1, activation='linear'))

# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Error cuadrático medio en el conjunto de prueba: {loss}')

# Crear gráficos
plt.figure(figsize=(12, 6))

# Gráfico de pérdida en entrenamiento y prueba
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Prueba')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Pérdida en entrenamiento y prueba')
plt.legend()

# Gráfico de predicciones vs. valores reales en el conjunto de prueba
plt.subplot(1, 2, 2)
y_pred = model.predict(X_test)
plt.scatter(y_test, y_pred)
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Predicciones vs. Valores Reales')

plt.tight_layout()
plt.show()